In [3]:
import pandas as pd
books = pd.read_csv("books_cleaned1.csv")

In [4]:
category_counts = books["categories"].value_counts()
categories_to_keep = category_counts[category_counts > 50].index
books = books[books["categories"].isin(categories_to_keep)]

In [5]:
books["categories"].value_counts()

categories
Fiction                      2075
Juvenile Fiction              377
Biography & Autobiography     302
History                       203
Literary Criticism            121
Comics & Graphic Novels       116
Philosophy                    112
Religion                      112
Drama                          81
Science                        55
Juvenile Nonfiction            51
Name: count, dtype: int64

In [6]:
category_mapping = { 'Fiction':"Fiction",
                    'Juvenile Fiction' : "Children's Fiction",
                    'Biography & Autobiography' : "Nonfiction",
                    'History':"Nonfiction",
                    'Literary Criticism':"Nonfiction",
                    'Philosophy':"Nonfiction",
                    'Religion':"Nonfiction",
                    'Comics & Graphic Novels': "Fiction",
                    'Drama' : "Fiction",
                    'Juvenile Nonfiction': "Children's Nonfiction",
                    'Science' : "Nonfiction",
                    'Poetry':"Fiction"
}

In [40]:
books["simple_categories"] = books["categories"].map(category_mapping)

In [43]:
books[~books["simple_categories"].isna()]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Children's Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,Taken at the Flood,9780007121014 A Few Weeks After Marrying An At...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5069,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",Fiction
5079,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5080,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5086,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [9]:
from transformers import pipeline

# Initialize zero-shot classification pipeline
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Example usage
candidate_labels = ["Fiction", "Nonfiction"]
sequence = "A memorable, mesmerizing heroine Jennifer -- beautiful, brilliant, rich, and famous."

result = zero_shot_classifier(sequence, candidate_labels)
print(result)

Device set to use cpu


{'sequence': 'A memorable, mesmerizing heroine Jennifer -- beautiful, brilliant, rich, and famous.', 'labels': ['Fiction', 'Nonfiction'], 'scores': [0.5435823798179626, 0.45641764998435974]}


In [10]:
sequence1 = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[0]

In [11]:
result = zero_shot_classifier(sequence1, candidate_labels)
print(result)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [12]:
import numpy as np

max_index = np.argmax( zero_shot_classifier(sequence1, candidate_labels)["scores"]) #to fetch the index with highest scores 
max_label =  zero_shot_classifier(sequence1, candidate_labels)["labels"][max_index] #label that comes on max_index returned
max_label



'Fiction'

In [13]:
#Zero shot classification over a description and return the label with the highest score
def generate_predictions(sequence , candidate_labels): #sequence - description of the book
    prediction =  zero_shot_classifier(sequence, candidate_labels)
    max_index =  np.argmax(prediction["scores"])
    max_label = prediction["labels"][max_index]
    return max_label


In [14]:
from tqdm import tqdm
actual_cats = []
predicted_cats =[]

for i in tqdm(range( 0 , 50)):
    sequence = books.loc[books["simple_categories"] == "Fiction" , "description"].reset_index(drop = True)[i]
    predicted_cats += [generate_predictions(sequence , candidate_labels)]
    actual_cats +=["Fiction"]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [03:55<00:00,  4.71s/it]


In [15]:
for i in tqdm(range( 0 , 50)):
    sequence = books.loc[books["simple_categories"] == "Nonfiction" , "description"].reset_index(drop = True)[i]
    predicted_cats += [generate_predictions(sequence , candidate_labels)]
    actual_cats +=["Nonfiction"]

100%|██████████| 50/50 [02:46<00:00,  3.33s/it]


In [16]:
predictions_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})

In [17]:
predictions_df

,actual_categories,predicted_categories
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction
...,...,...
95,Nonfiction,Nonfiction
96,Nonfiction,Nonfiction
97,Nonfiction,Nonfiction
98,Nonfiction,Fiction


In [18]:
predictions_df["correct_predictions"] = (np.where(predictions_df["actual_categories"]==predictions_df["predicted_categories"],1,0))

In [19]:
accuracy = predictions_df["correct_predictions"].sum()/len(predictions_df)

In [20]:
accuracy * 100

np.float64(82.0)

In [36]:
isbns = []
predictions = []

missing_cats = books.loc[books["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop = True)

In [37]:
missing_cats

,isbn13,description


In [44]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats += [generate_predictions(sequence, candidate_labels)]
    isbns += [missing_cats.loc["isbn13"][i]]

0it [00:00, ?it/s]


In [45]:
books[books["categories"].str.lower().isin(
    ["romance",
     "science fiction",
     "scifi",
     "horror",
     "fantasy",
     "thriller",
     "comedy",
     "crime",
     "historical",
     "mystery"]
)]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories


In [46]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Children's Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,Taken at the Flood,9780007121014 A Few Weeks After Marrying An At...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5069,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",Fiction
5079,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5080,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5086,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [48]:
books.to_csv("books_with_categories.csv", index = False)